# Week 41 - Exercises

OLS REGRESSION

1. Plain GD with fixed learning rate

In [18]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from random import random, seed
from autograd import grad

np.random.seed(2018)

In [5]:
n = 100
x = np.random.rand(n, 1)
y = 4 + 3* x + 2*x**2 + np.random.randn(n, 1)
X = np.c_[np.ones((n, 1)), x, x**2]

XT_X = X.T @ X
theta_linreg = np.linalg.inv(X.T @ X) @ (X.T @ y)
print("Theta from own inversion OLS:", theta_linreg)
H = (2.0/n)*XT_X
EigValues, EigVectors = np.linalg.eig(H)

theta = np.random.randn(3, 1)
eta = 1.0/np.max(EigValues)
num_iter = 1000

for iter in range(num_iter):
    gradients = 2.0/n* X.T @ ((X @ theta) - y)
    theta -= eta*gradients
print("Theta from GD:", theta)

Theta from own inversion: [[3.92107297]
 [3.94878834]
 [1.23008291]]
Theta from GD: [[3.93072664]
 [3.89317344]
 [1.28332218]]


2. Plain GD with fixed learning rate and momentum

In [9]:
change = 0.0
delta_momentum = 0.3

for iter in range(num_iter):
    gradients = 2.0/n* X.T @ ((X @ theta) - y)
    new_change = eta*gradients + delta_momentum*change
    theta -= new_change
    change = new_change
print("Theta from GD with momentum:", theta)

Theta from GD with momentum: [[4.11202724]
 [2.95196881]
 [2.14193576]]


Comparison:

3. Stochastic gradient descent 

In [8]:
num_epochs = 100
M = 5
m = int(n/M)
t0, t1 = 5, 50

def learning_schedule(t):
    return t0/(t+t1)

theta = np.random.randn(3, 1)

for epoch in range(num_epochs):
    for i in range(m):
        rand_index = M+np.random.randint(m)
        x_i = X[rand_index:rand_index+M]
        y_i = y[rand_index:rand_index+M]
        gradients = (2.0/M)*x_i.T @ ((x_i @ theta) - y_i)
        eta = learning_schedule(epoch*m + i)
        theta -= eta*gradients
print("Theta from SGD:", theta)

Theta from SGD: [[4.2098165 ]
 [2.94005582]
 [2.14982706]]


Discussion:

4a. Adagrad without and with momentum for GD

4a. Adagrad without and with momentum for SGD

5. RMSprop and Adam

In [29]:
# RMSprop
def CostOLS(y, X, theta):
    return np.sum((y - X @ theta)**2)

training_gradient = grad(CostOLS, 2)
theta = np.random.randn(3, 1)
num_epochs = 100
M = 5
m = int(n/M)
eta = 0.01
rho = 0.90
delta = 1e-8

for epoch in range(num_epochs):
    Giter = 0.0
    for i in range(m):
        rand_index = M+np.random.randint(m)
        x_i = X[rand_index:rand_index+M]
        y_i = y[rand_index:rand_index+M]
        gradients = (1.0/M)*training_gradient(y_i, x_i, theta)
        Giter = (rho*Giter + (1-rho)*gradients*gradients)
        update = gradients*eta/(delta + np.sqrt(Giter))
        theta -= update
print("Theta from SGD with RMSprop:", theta)

Theta from SGD with RMSprop: [[3.78533369]
 [3.53362961]
 [1.95202195]]


In [43]:
# Adam
training_gradient = grad(CostOLS, 2)
theta = np.random.randn(3, 1)
num_epochs = 100
M = 5
m = int(n/M)
eta = 0.01
beta1 = 0.9
beta2 = 0.999
delta = 1e-7

iter = 0
for epoch in range(num_epochs):
    first_moment = 0.0
    second_moment = 0.0
    iter += 1
    for i in range(m):
        rand_index = M+np.random.randint(m)
        x_i = X[rand_index:rand_index+M]
        y_i = y[rand_index:rand_index+M]
        gradients = (1.0/M)*training_gradient(y_i, x_i, theta)
        first_moment = beta1*first_moment + (1-beta1)*gradients
        second_moment = beta2*second_moment + (1-beta2)*gradients*gradients
        first_term = first_moment/(1.0-beta1**iter)
        second_term = second_moment/(1.0-beta2**iter)
        update = eta*first_term/(np.sqrt(second_term)+delta)
        theta -= update
print("Theta from SGD with Adam:", theta)

Theta from SGD with Adam: [[4.04005989]
 [2.9419719 ]
 [2.366378  ]]


RIDGE REGRESSION

In [17]:
n = 100
x = np.random.rand(n, 1)
y = 4 + 3* x + 2*x**2 + np.random.randn(n, 1)
X = np.c_[np.ones((n, 1)), x, x**2]
lmb = np.logspace(-4, 4, 10)

thetaa = np.zeros(len(lmb))
etaa = np.zeros(len(lmb))
XT_X = X.T @ X
for j in range(len(lmb)):
    theta_ridge = np.linalg.inv(X.T @ X + lmb[j]*np.eye(len(X.T))) @ (X.T @ y)
    print("Theta from own inversion Ridge:", theta_ridge)
    H = (2.0/n)*XT_X
    EigValues, EigVectors = np.linalg.eig(H)

    theta = np.random.randn(3, 1)
    eta = 1.0/np.max(EigValues)
    num_iter = 1000

    for iter in range(num_iter):
        gradients = 2.0/n* X.T @ ((X @ theta) - y)
        theta -= eta*gradients
    print("Theta from GD:", theta)

Theta from own inversion Ridge: [[ 3.24671607]
 [ 6.53895183]
 [-1.01339229]]
Theta from GD: [[ 3.34338792]
 [ 5.93070167]
 [-0.40819955]]
Theta from own inversion Ridge: [[ 3.24817945]
 [ 6.52976414]
 [-1.004334  ]]
Theta from GD: [[ 3.34463937]
 [ 5.9228277 ]
 [-0.40036532]]
Theta from own inversion Ridge: [[ 3.25925695]
 [ 6.4602182 ]
 [-0.93578167]]
Theta from GD: [[ 3.33416567]
 [ 5.98872691]
 [-0.46593195]]
Theta from own inversion Ridge: [[ 3.33218031]
 [ 6.00254133]
 [-0.48537675]]
Theta from GD: [[ 3.3290785 ]
 [ 6.02073475]
 [-0.49777824]]
Theta from own inversion Ridge: [[3.57497191]
 [4.48200865]
 [0.989849  ]]
Theta from GD: [[ 3.32059214]
 [ 6.07412988]
 [-0.5509039 ]]
Theta from own inversion Ridge: [[3.77378748]
 [3.19813232]
 [2.01500249]]
Theta from GD: [[ 3.33033688]
 [ 6.01281719]
 [-0.48990065]]
Theta from own inversion Ridge: [[3.58538042]
 [2.41464775]
 [1.7710085 ]]
Theta from GD: [[ 3.33237162]
 [ 6.00001489]
 [-0.47716295]]
Theta from own inversion Ridge: [[1.